In [29]:
#import data

# -*- coding: utf-8 -*-

#import data
from datetime import datetime
import pandas as pd
import numpy as np
import torch.onnx
import os

import torch
import torch.nn as nn
# import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import datasets, transforms



In [30]:
all_x_data = []
all_y_data = []

file_path = os.path.join("..","dataset","traindata","drive_data_train.csv")
file_path = os.path.abspath(file_path)
print("===trin data file path===", "\n", file_path)

# Pandas를 이용해 파일 읽기
data = pd.read_csv(file_path, sep=",", header=0, dtype=float)
x_data = torch.tensor(data.iloc[:, [0,2,3,4,5]].values, dtype=torch.float32)
y_data = torch.tensor(data.iloc[:, 1:2].values, dtype=torch.float32)

# 전체 데이터 리스트에 추가
all_x_data.append(x_data)
all_y_data.append(y_data)
        
# 리스트에 저장된 모든 데이터를 하나의 텐서로 결합
all_x_data = torch.cat(all_x_data, dim=0)
all_y_data = torch.cat(all_y_data, dim=0)

# 전체 데이터에 대한 평균과 분산 계산
mean_x = torch.mean(all_x_data, dim=0)
std_x = torch.std(all_x_data, dim=0)

Max_Vy = max(abs(all_y_data))

===trin data file path=== 
 c:\github_JM\2024-Neural-networks-and-deep-learning-teamproject\dataset\traindata\drive_data_train.csv


In [31]:
x_data = (all_x_data - mean_x) / std_x  # mean과 std는 훈련 데이터셋에서 계산된 값 사용
y_data = all_y_data/float(Max_Vy)

# 배치사이즈 설정
batch_size = 512

In [32]:
class My_dataset(Dataset):
    def __init__(self, x_data, y_data, transform=None):  #데이터 셋 선처리 해주는 부분
        self.x_data = x_data.unsqueeze(1)
        self.y_data = y_data
        self.transform = transform
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):
        x = self.x_data[index]  # (1, input_size) 형식
        y = self.y_data[index]  # (1) 형식

        if self.transform:
            x, y = self.transform((x, y))
        return x, y

    def __len__(self):
        return self.len

class ToTensor:
    def __call__(self, sample):
        x, y = sample
        x = torch.FloatTensor(x)
        y = torch.FloatTensor(y)
        return x, y


# 데이터 로드
trans = transforms.Compose([ToTensor()])

train_data = My_dataset(x_data, y_data, transform=trans)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, drop_last=True)

In [33]:
# LSTM 모델 정의
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # 마지막 시점의 출력만 사용
        out = self.tanh(out)  # tanh 활성화 함수 적용
        return out
    
    
# 모델 초기화 및 손실 함수, 옵티마이저 설정
input_size = 5
hidden_size = 64
num_layers = 2
output_size = 1

model = LSTMNet(input_size, hidden_size, num_layers, output_size)

In [34]:
Loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.00005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.99)

In [35]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total_params)

51521


In [36]:
now = datetime.now()
formatted_time = now.strftime("%Y_%m_%d_%H_%M")

model_path = os.path.join("model","LSTM_model_{0}.pt".format(formatted_time))
model_path = os.path.abspath(model_path)
print("===trained model path===", "\n", model_path)

losses = []

# 훈련 루프
num_epochs = 600
num_data = len(x_data)


min_loss = 50
training_epoch = 0

try:
    for epoch in range(num_epochs):
        losses = []
        epoch_losses = []
        average_losses = []

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            
            inputs = inputs  # LSTM 입력 형태에 맞게 변경 (batch_size, seq_len, input_size)
            outputs = model(inputs)
            targets = targets.view(-1, 1)
            
            loss = Loss(outputs, targets)
            loss.backward()
            
            optimizer.step()
            epoch_losses.append(loss.item())
    
        average_losses = np.mean(epoch_losses)
        losses.append(average_losses)
    
        if average_losses < min_loss:
            min_loss = average_losses
            model.eval()  # 모델 평가 모드로 전환
            x_dummy = torch.randn(1, 1, input_size)
            traced_model = torch.jit.trace(model, x_dummy)
            traced1_model_path = model_path[:-3] + '_Traced_1.pt'
            torch.jit.save(traced_model, traced1_model_path)

        if epoch in [100,200,300,400]:
            model.eval()  # 모델을 평가 모드로 설정
            x_dummy = torch.randn(1, 1, input_size)
            traced_model = torch.jit.trace(model, x_dummy)
            evaluated_model_path = model_path[:-3] + '_Evaluated_' + str(epoch) + '.pt'
            torch.jit.save(traced_model, evaluated_model_path)

        # Scheduler 업데이트
        scheduler.step()
        training_epoch = epoch
        # 진행 상황 출력 (선택적)
        print(f"Epoch {epoch}, average_losses: {average_losses}")        

except KeyboardInterrupt:
    print("Training interrupted. Saving the model.")
    model.eval()  # 모델을 평가 모드로 설정
    x_dummy = torch.randn(1, 1, input_size)
    traced_model = torch.jit.trace(model, x_dummy)
    # 트레이싱된 모델 저장
    traced2_model_path = model_path[:-3] + '_Traced_2.pt'
    torch.jit.save(traced_model, traced2_model_path)

# 모델 저장
model.eval()  # 모델을 평가 모드로 설정
x_dummy = torch.randn(1, 1, input_size)
traced_model = torch.jit.trace(model, x_dummy)

# 트레이싱된 모델 저장
traced3_model_path = model_path[:-3] + '_Traced_3.pt'
torch.jit.save(traced_model, traced3_model_path)
print("Traced model saved.")

===trained model path=== 
 c:\github_JM\2024-Neural-networks-and-deep-learning-teamproject\LSTM\model\LSTM_model_2024_06_05_19_44.pt
Epoch 0, average_losses: 0.02028804910127827
Epoch 1, average_losses: 0.01778512915796669
Epoch 2, average_losses: 0.015141648871076005
Training interrupted. Saving the model.
Traced model saved.
